# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [7]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2025-04-27 19:55:55--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   163MB/s    in 0.3s    

2025-04-27 19:55:56 (163 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
replace imdb-reviews-pt-br.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [8]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [9]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [10]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

Nesta questão, você irá calcular a soma dos IDs para entradas onde o sentimento ('sentiment') é 'neg'.

### Objetivo:
- Usar a coluna 'sentiment' como chave e somar os valores da coluna 'id'.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

A função map irá transformar cada linha do dataframe em uma **tupla** (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: coluna 'id' convertida para inteiro.

In [11]:
def map1(x):
    return (x['sentiment'], int(x['id']))

## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

A função reduce irá somar os valores dos IDs agrupados por chave ('sentiment').

In [12]:
def reduceByKey1(x, y):
    return x + y

## Aplicação do map/reduce e visualização do resultado

Aqui, você aplicará as funções de map e reduce ao dataframe Spark para calcular os resultados. Não se esqueça de usar o método `.collect()` para visualizar os resultados.

In [14]:
resultado = imdb_df.rdd.map(map1).reduceByKey(reduceByKey1).collect()

for chave, valor in resultado:
    if chave == "neg":
        print(f"Meu RU: 4470123 - Soma dos IDs com sentimento NEG: {valor}")


Meu RU: 4470123 - Soma dos IDs com sentimento NEG: 459568555


# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave de uma tupla principal e como valor uma outra tupla com a soma das palavras de cada idioma como valor.

A função map irá transformar cada linha do dataframe em uma tupla (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: Nova tupla com:
  - Elemento 0: soma das palavras da coluna 'text_en'
  - Elemento 1: soma das palavras da coluna 'text_pt'

OU
- Chave: coluna 'sentiment'
- Valor: (soma das palavras da coluna 'text_pt') - (soma das palavras da coluna 'text_en')
  

Para contar as palavras deve-se primeiro separar os textos em uma lista de palavras para então descobrir o tamanho desta lista.
Dicas:

1. Use o método .split() e não .split(" ") de string para separar as palavras em uma lista ou use a função split(coluna de texto, regex) do pyspark com o regex igual à "[ ]+" ou "\s+"
2. Use len() para descobrir o tamanho da lista de palavras.

In [15]:
def map2(x):
    if x['sentiment'] == 'neg':
        palavras_en = len(x['text_en'].split()) if x['text_en'] else 0
        palavras_pt = len(x['text_pt'].split()) if x['text_pt'] else 0
        return ('neg', (palavras_en, palavras_pt))
    else:
        return ('outro', (0, 0))

## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment" (dependerá de como você optou por fazer sua função map2).

A função reduce irá somar os valores das quantidades de palavras agrupados por chave ('sentiment').

In [16]:
def reduceByKey2(x, y):
    return (x[0] + y[0], x[1] + y[1])

## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [17]:
resultado = imdb_df.rdd.map(map2).reduceByKey(reduceByKey2).collect()

for chave, valor in resultado:
    if chave == "neg":
        palavras_en_total, palavras_pt_total = valor
        diferenca = palavras_pt_total - palavras_en_total
        print(f"Meu RU: 4470123 - Diferença de palavras (PT - EN) para NEG: {diferenca}")

Meu RU: 4470123 - Diferença de palavras (PT - EN) para NEG: 54949
